In [1]:
import pandas as pd
import numpy as np

def function(path):
    dataset = pd.read_csv(path)
    return dataset

In [2]:
df = function('pks.csv')

In [22]:
df = df.drop('name', axis = 1)

In [37]:
df.to_csv("pkson.csv",index=False)

In [17]:
#cols = list(df.columns.values) #Make a list of all of the columns in the df
#cols.pop(cols.index('PPE')) #Remove b from list
#cols.pop(cols.index('status')) #Remove x from list
#df = df[cols+['PPE','status']] #Create new dataframe with columns in the order you want

In [22]:
#df

In [24]:
from sklearn.model_selection import train_test_split

def function1(dataset):
    X = dataset.drop(dataset.columns[-1], axis=1)
    y = dataset[dataset.columns[-1]].astype(int)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    return [X_train, X_test, y_train, y_test]

In [25]:
output1 = function1(df)

In [26]:
#output1

In [27]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE, RFECV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit, cross_validate
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector,make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
#from autokeras import StructuredDataClassifier
def function2(X):
    imp_median = SimpleImputer(strategy='median', add_indicator=True)
    scaler = StandardScaler()

    # set up preprocessing numeric columns
    imp_median = SimpleImputer(strategy='median', add_indicator=True)
    scaler = StandardScaler()

    # set up preprocessing categorical columns
    imp_constant = SimpleImputer(strategy='constant')
    ohe = OneHotEncoder(handle_unknown='ignore')

    # select columns by datatype
    num_cols = make_column_selector(dtype_include='number')
    cat_cols = make_column_selector(dtype_exclude='number')

    # do all preprocessing
    preprocessor = make_column_transformer(
        (make_pipeline(imp_median, scaler), num_cols),
        (make_pipeline(imp_constant, ohe), cat_cols)
    )
    
    mlmodel = LogisticRegression()
    models = dict()
    for i in range(2, len(X.columns)):
        rfe = RFE(estimator=mlmodel, n_features_to_select=i)
        pipeline = make_pipeline(preprocessor,rfe)
        #print(pipeline.named_steps['rfe'].ranking_[i],pipeline.named_steps['rfe'].support_[i])
        models[str(i)] = pipeline
    return [models]

In [28]:
output2=function2(output1[0])

In [29]:
#output2[0]

In [30]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE, RFECV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline

# evaluate a give model using cross-validation
def function3(models, X_train, y_train):
    scores=[]
    for i in models:
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
        score = cross_val_score(models[i], X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        scores.append(score.mean())
    tmp = max(scores)
    index = scores.index(tmp)
    print(index)
    return models[str(index)]

In [31]:
output_final = function3(output2[0],output1[0],output1[2])

18


In [32]:
output_final

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D981100820>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='constant')),
                                              

In [33]:
datasetout = df

In [34]:
select_pipem=output_final
m=select_pipem.fit(datasetout.drop(datasetout.columns[-1], axis=1),datasetout[datasetout.columns[-1]].astype(int))
k=select_pipem.named_steps['rfe'].support_,select_pipem.named_steps['rfe'].ranking_

In [35]:
listn=''
i=0
flag=0
clm=datasetout.columns[:-1]
for ik in k[1]:
    #print(i,ik, k[1])
    if ik == 1:
        if flag ==0:
            listn=clm[i]
            flag = 1
        else:
            listn=listn+'.'+clm[i]
    i=i+1

In [36]:
k

(array([ True,  True, False,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True, False,  True,  True,  True, False,
         True,  True,  True,  True]),
 array([1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 1]))